In [ ]:
import pandas as pd
import numpy as np
from tkinter import *
from tkinter import ttk
from pandas.io.formats.style import Styler

##### STEP 0 #####

### setting file paths

print("Please enter the following inputs Filenames with the full path ...")

# order_file = input(r'FILE  _Order_Tracking_Report-_DXC_-Queensland_Rail :: ')
order_file = r"C:\DEB\DXC_Automation\UCMS_31107\279577_Order_Tracking_Report-_DXC_-Queensland_Rail.xls"
# dashboard_file = input(r'FILE  Daily dashboard :: ')
dashboard_file = r"C:\DEB\DXC_Automation\UCMS_31107\Daily dashboard.xlsx"
# lease_supl_cons_file = input(r'FILE  DXC Lease Supplements Consolidated :: ')
lease_supl_cons_file = r"C:\DEB\DXC_Automation\UCMS_31107\DXC Lease Supplements Consolidated.xls"
# monthly_rental_file = input(r'FILE  Monthly rental prize :: ')
monthly_rental_file = r"C:\DEB\DXC_Automation\UCMS_31107\Monthly rental prize.xlsx"
#output_file = 

### setting up the log file 


In [ ]:
df_Output_File = pd.DataFrame(columns=["Country","Customer","Contract Vendor","Contract Term","Billing Cycle","Contract Equipment Cost IBU","Contract Lease Payment","Order Number","Serial Number","Need Asset Tag RE","Repository State","Repository Sub State","Installation Date","Need PO RE","Asset Description","PO Number","Order Qty","Order Qty Reconciled","Net Price","Rental per Asset","Reconcilied"], index=None)
df_Output_File.info()


In [ ]:
'''
##### STEP 1 #####
In this step Order Tracking file data would be processed - Serial Numbers would be splitted to multiple rows and then 2 filters are applied based on Product Description and Brand Description.
The resulting dataframe would be used as the referrence DF for the final Output File
''' 
# os.path.join()
df_orders = pd.read_excel(order_file, sheet_name="Order Status Data", header=0, usecols=["Po Num", "Order Num", "Product Desc", "Brand Desc", "Combine svc tag", "Order Qty"], encoding="ISO-8859-1", index_col=None, engine=None, na_values=None)
df_orders["Order Qty Reconciled"] = []

row_start = 0
row_end = df_orders.shape[0]    
total_po_nums = total_order_numbers = total_serial_numbers = 0
po_nums_missing = order_nums_missing = serial_numbers_missing = 0

df_new_rows = pd.DataFrame(columns=["Po Num", "Order Num", "Product Desc", "Brand Desc", "Combine svc tag", 'Order Qty', 'Order Qty Reconciled'])
for each_row in range(row_start,row_end):
    po_num = df_orders.iloc[each_row,0]
    order_num = df_orders.iloc[each_row,1]
    serial_numbers = df_orders.iloc[each_row,4]
    # print(len(df_orders.iloc[66,4].split(sep=",")))
    if (isinstance(po_num, np.int64)):
        total_po_nums += 1
    else:
        po_nums_missing += 1
        
    if (isinstance(order_num, np.int64)):
        total_order_numbers += 1
    else:
        order_nums_missing +=1
    
    if (isinstance(serial_numbers, str)):
        serial_numbers_count = len(serial_numbers.split(sep=","))
        total_serial_numbers = total_serial_numbers + serial_numbers_count
        if serial_numbers_count >1:
            if serial_numbers_count != df_orders.iloc[each_row,5]:
                df_orders.iloc[each_row,6] = "ORDER-SUPPLY MISMATCH for this PO"
            else:
                df_orders.iloc[each_row,6] = "Supply as per Order"

            serial_numbers_list = serial_numbers.split(sep=",") ## convert the serial number chunks to a list
            ## replace the cell with the first serial number
            serial_num_first_one = tuple(serial_numbers_list)[0]
            df_orders.iloc[each_row,4] = serial_num_first_one
            ## now the other serial numbers
            serial_num_next_ones = tuple(serial_numbers_list)[1:]
#             print(serial_num_first_one)
#             print(serial_num_next_ones)
#             print("="*20)
            ## preparing the list of values for the new row
            cnt = 0
            for next_serials in range(0, len(serial_num_next_ones)):
                new_list = list()
                new_list.append(df_orders.iloc[each_row,0])
                new_list.append(df_orders.iloc[each_row,1])
                new_list.append(df_orders.iloc[each_row,2])
                new_list.append(df_orders.iloc[each_row,3])
                new_list.append(serial_num_next_ones[next_serials])                            
                new_list.append(df_orders.iloc[each_row,5])
                cnt += 1
#                 print(next_serials, serial_num_next_ones[next_serials])
            
            ## converting the list to a Series
                temp_series = pd.Series(new_list, name="MoreSerials", index = df_orders.columns)
                df_new_rows = df_new_rows.append(temp_series, ignore_index=True)
                
#         print("{} added".format(cnt))

    else:       
        serial_numbers_missing +=1
        
'''    
print("Order Tracking Report before unpacking has - {} - rows".format(df_orders.shape[0]))
print("Total number of Po numbers found - {}".format(total_po_nums))
print("Total number of Po numbers missing - {}".format(po_nums_missing))
print("Total number of Order numbers found - {}".format(total_order_numbers))
print("Total number of Order numbers missing - {}".format(order_nums_missing))
print("Total number of serial numbers found - {}".format(total_serial_numbers))
print("Total number of serial numbers missing - {}".format(serial_numbers_missing))
'''
# print(df_orders.shape)
df_orders = df_orders.append(df_new_rows, ignore_index=True) 
''' merging the new rows with the additional serial numbers to the main DF'''
# print(df_orders.shape)
print("Total rows after unpacking the Serial numbers - {}".format(df_orders.shape[0]))
### Order Tracking file dataframe prepared
### Applying filters as per the request
df_orders_filter_F = df_orders[(df_orders['Product Desc'] != "CLIENT PERIPHERALS") & (df_orders['Product Desc'] != "SUPPORT SERVICES")]  
# print(df_orders_filter_F.shape)
print("Removed {} rows for Product Description as CLIENT PERIPHERALS and SUPPORT SERVICES".format(df_orders.shape[0] - df_orders_filter_F.shape[0]))
df_orders_filter_G = df_orders_filter_F[(df_orders_filter_F['Brand Desc'] != "PARTNER") & (df_orders_filter_F['Brand Desc'] != "UNKNOWN" )]
# print(df_orders_filter_G.shape)
print("Removed {} rows for Brand Description as PARTNER and UNKNOWN".format(df_orders_filter_F.shape[0] - df_orders_filter_G.shape[0]))
df_orders_filtered = df_orders_filter_G
print("Valid rows after applying filters - {}".format(df_orders_filtered.shape[0]))
### end of processing Order Tracking file
# df_Output_File = pd.DataFrame()
df_Output_File["Order Number"] = df_orders_filtered["Order Num"]
df_Output_File["Serial Number"] = df_orders_filtered["Combine svc tag"]
df_Output_File["PO Number"] = df_orders_filtered["Po Num"]
df_Output_File["Order Qty"] = df_orders_filtered["Order Qty"]

# df_Output_File
df_orders

In [ ]:
##### STEP 2 #####

df_daily_dashboard = pd.read_excel(dashboard_file, sheet_name="Page 1", header=0, usecols=["Asset tag", "Serial number", "State", "Substate", "Assigned to", "Email", "Model", "Installed Date", "PO number"], encoding="ISO-8859-1", index_col=None, engine=None, na_values=None)

# print(df_daily_dashboard)
# print(df_daily_dashboard.shape[0])
# cnt = 0
serial_num_missing_from_dashboard = list()
serial_num_match_with_dashboard = list()
po_num_mismatch = list()
po_num_matched = list()
for row_idx_ in range(0, df_orders_filtered.shape[0]):  #looping the Order Tracking Report dataframe'''
    serial_ordered = df_orders_filtered.iloc[row_idx_,4]  '''capture only the serial number'''
#     print(serial_ordered)
    serial_num_Flag = True ''' setting up a flag variable '''
    counter = df_daily_dashboard.shape[0] - 1 '''setting a looping variable to the last row to check from the Dashboard DF'''
    while (counter >= 0): '''looping back'''
#         print(df_daily_dashboard.iloc[counter, 1])
        if (df_daily_dashboard.iloc[counter, 1] == serial_ordered):
            serial_num_Flag = True
            serial_num_match_with_dashboard.append(row_idx_)
            df_Output_File[]
#             print("sr matched")
            break
        else:
            serial_num_Flag = False    
        counter -= 1
#     print(counter)
    if (serial_num_Flag == False):
        serial_num_missing_from_dashboard.append(row_idx_)
    else:
        if (df_daily_dashboard.iloc[counter, 8] != str(df_orders_filtered.iloc[row_idx_, 0])):
            po_num_mismatch.append(counter)
        else:
            po_num_matched.append(counter)
#     print("-"*20)
    
print(len(serial_num_missing_from_dashboard))
print(len(serial_num_match_with_dashboard))
print(len(po_num_mismatch))
print(len(po_num_matched))
''' coping the required columns to the Output File DF '''
df_Output_File[""] = df_daily_dashboard[""]

# for po_ in po_num_mismatch:
#     df_daily_dashboard.iloc[po_,8].style.applymap(color='darkorange')

    df_daily_dashboard.head()

In [ ]:

##### STEP 3 #####

df_DXC_Lease_Supl_Cons= pd.read_excel(lease_supl_cons_file, sheet_name="Asset List", encoding="ISO-8859-1", index_col=None, engine=None, na_values=None)
    ## removing the top blank rows
df_DXC_Lease_Supl_Cons= df_DXC_Lease_Supl_Cons[9:]
df_DXC_Lease_Supl_Cons= df_DXC_Lease_Supl_Cons.reset_index(drop = True)
    ## resetting the start row
df_DXC_Lease_Supl_Cons.columns = df_DXC_Lease_Supl_Cons.iloc[0]
    ## resetting the start row of the data
df_DXC_Lease_Supl_Cons= df_DXC_Lease_Supl_Cons[1:]
    ## dropping all completely blank rows and retain the same dataframe
df_DXC_Lease_Supl_Cons.dropna(how = 'all', inplace=True)
    ## Unpacking the last column with merged cells accross column
df_DXC_Lease_Supl_Cons["Lease Suppliment"].fillna(method = 'ffill', inplace=True)
    ## dropping more unnecessary rows having "EQUIPMENT DESCRIPTION"
df_DXC_Lease_Supl_Cons.dropna(how = 'any', inplace=True)
print("Total count of Invoice Number rows - {}".format(df_DXC_Lease_Supl_Cons.shape[0]))
# print(df_DXC_Lease_Supl_Cons.shape)
# df_DXC_Lease_Supl_Cons.iloc[80:]

    ### Iterating the dataframe for splitting the "Tag Number" column (Serial Number). 
    ### This column has seperator as 'space', 'new line', 'carriage return' and line endigs with 'new line'
start_row = 0
last_row = df_DXC_Lease_Supl_Cons.shape[0]
tag_nums_each_row  = tag_num_total_count = 0
df_new_rows = pd.DataFrame().reindex(columns=df_DXC_Lease_Supl_Cons.columns)
# print(df_temp)
# print(type(df_temp))
# print(df_temp.shape)

for rows_ in range(start_row, last_row):
    tags = df_DXC_Lease_Supl_Cons.iloc[rows_,6]
    tags_1 =  tags.replace("\n"," ")  ## removing 'new line'
    tags_2 = tags_1.replace("\r","")  ## removing 'carriage return'
    tag_list = tags_2.split(sep = " ")  ## string to list -> delimiter = 'space'
    tag_list = list(filter(None, tag_list)) ## removing empty list entries formed due to the previous processings
#     print(tag_list, len(tag_list))
#     print("-"*30)
    tag_nums_each_row = len(tag_list)  
    tag_num_total_count = tag_num_total_count + tag_nums_each_row
    
#     print("{} more tag/serial numbers found. A total of {} tag/serial numbers tracked so far..".format(tag_nums_each_row, tag_num_total_count))
    if tag_nums_each_row > 1:
        df_DXC_Lease_Supl_Cons.iloc[rows_,6] = tag_list[0]
        tag_list_tail = tag_list[1:]
        cnt = 0
        for expand_rows_ in range(0, len(tag_list_tail)):
            new_list = list()
            new_list.append(df_DXC_Lease_Supl_Cons.iloc[rows_,0])
            new_list.append(df_DXC_Lease_Supl_Cons.iloc[rows_,1])
            new_list.append(df_DXC_Lease_Supl_Cons.iloc[rows_,2])
            new_list.append(df_DXC_Lease_Supl_Cons.iloc[rows_,3])
            new_list.append(df_DXC_Lease_Supl_Cons.iloc[rows_,4])
            new_list.append(df_DXC_Lease_Supl_Cons.iloc[rows_,5])
            new_list.append(tag_list_tail[expand_rows_])
            new_list.append(df_DXC_Lease_Supl_Cons.iloc[rows_,7])
            temp_series = pd.Series(new_list, name="MoreTags", index = df_DXC_Lease_Supl_Cons.columns )
            df_new_rows = df_new_rows.append(temp_series, ignore_index=True)
            cnt += 1
#         print("{} more new rows added ... ".format(cnt))
#     print("{} total new rows added".format(df_new_rows.shape[0]))
print("A total of {} tag/serial numbers have been tracked.".format(tag_num_total_count))
df_DXC_Lease_Supl_Cons= df_DXC_Lease_Supl_Cons.append(df_new_rows, ignore_index=True)
print("Post processing a total of {} new rows added".format(df_new_rows.shape[0]))
print("Final size of the Lease Supplement dataframe - {} rows...".format(df_DXC_Lease_Supl_Cons.shape[0]))
        

In [ ]:
##### STEP 4 #####

df_monthly_rental_prize =  pd.read_excel(monthly_rental_file, sheet_name="Sheet1", header=0, encoding="ISO-8859-1", index_col=None, engine=None, na_values=None)
print(df_monthly_rental_prize)
